For the TYC-3023-1974-1 and RZ Cep datasets

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Results/result_cep_corr.csv')

In [ ]:
# Ensure the "Date" column is in datetime format
data["Date"] = pd.to_datetime(data["Date"], format="%Y-%m-%dT%H:%M:%S.%f")

# List of bands to create separate DataFrames
bands = ["R", "G", "B"]

# Create a dictionary to hold DataFrames for each band and drop rows with missing apparent magnitudes
band_dfs = {}
for band in bands:
    band_cols = ["Object", "Date", f"Apparent magnitude {band} (instrumental)"]
    band_dfs[band] = data[band_cols].dropna(subset=[f"Apparent magnitude {band} (instrumental)"])

# Function to correct a band's DataFrame
def correct_band_dataframe(df, band):
    # Find the reference date for this band
    object_count_per_date = df.groupby("Date")["Object"].nunique()
    reference_date = object_count_per_date.idxmax()
    
    # Initialize a DataFrame to hold the average magnitude differences
    avg_mag_diff = pd.DataFrame(columns=["Date", "Avg_Magnitude_Difference"])
    
    # Loop through each unique date in this band's DataFrame
    unique_dates = df["Date"].unique()
    for date in unique_dates:
        if date == reference_date:
            continue  # Skip the reference date itself
        
        # Find common objects between the reference date and the current date
        ref_mag_col = f"Apparent magnitude {band} (instrumental)"
        common_stars = df[df["Date"] == reference_date][["Object", ref_mag_col]].merge(
            df[df["Date"] == date][["Object", ref_mag_col]], on="Object", suffixes=("_ref", "_other")
        )
        
        if common_stars.empty:
            # If no common stars, set average difference to zero
            avg_mag_diff = avg_mag_diff.append({
                "Date": date,
                "Avg_Magnitude_Difference": 0.0
            }, ignore_index=True)
        else:
            # Calculate the average magnitude difference
            mag_difference = common_stars[f"{ref_mag_col}_ref"] - common_stars[f"{ref_mag_col}_other"]
            avg_diff = mag_difference.mean()
            avg_mag_diff = avg_mag_diff.append({
                "Date": date,
                "Avg_Magnitude_Difference": avg_diff
            }, ignore_index=True)

    # Adjust magnitudes for the current band
    new_mag_col = f"Corrected magnitude {band} (instrumental)"
    for date in unique_dates:
        if date == reference_date:
            continue
        
        original_mag_col = f"Apparent magnitude {band} (instrumental)"
        row = avg_mag_diff[avg_mag_diff["Date"] == date]
        
        avg_diff = 0.0
        if not row.empty:
            avg_diff = row["Avg_Magnitude_Difference"].values[0]
        
        df.loc[df["Date"] == date, new_mag_col] = df.loc[df["Date"] == date, original_mag_col] + avg_diff
    df[df["Date"] == reference_date][f"Corrected magnitude {band} (instrumental)"] = df[df["Date"] == reference_date][f"Apparent magnitude {band} (instrumental)"]
    df.drop(f"Apparent magnitude {band} (instrumental)", axis=1, inplace=True)
    return df

# correct each band's DataFrame
for band in bands:
    band_dfs[band] = correct_band_dataframe(band_dfs[band], band)

# Join the correctd DataFrames based on "Date" and "Object"
joined_df = band_dfs["R"]
for band in ["G", "B"]:
    joined_df = joined_df.merge(band_dfs[band], on=["Object", "Date"], how="outer")

In [ ]:
final_df = data.merge(joined_df, on=["Object", "Date"], how="outer")
final_df.to_csv('Results/result_cep_corr_1.csv', index=False)